<h1>QA Dataset Generation</h1>

In [ ]:
!pip install transformers=="4.25.1" sentencepiece=="0.1.97" utoken=="0.1.8" nltk=="3.8.1" datasets=="2.8.0" torch=="1.13.1+cu116" numpy=="1.21.6" tqdm=="4.64.1" --quiet

In [ ]:
# import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
import json
from pathlib import Path
import torch
from torch.utils.data import DataLoader
import time
import os
import random

## Model

In [ ]:
question_generator_args=dict(
    qg_model="./QGmodel/QGmodelfinal2",
    qa_model="./QA_models/banglabert-finetuned-squadbangla1-final",
    output_dir="./generated_questions",
    max_q_length=64,
    temperature=1.0,
    top_k=30,
    top_p=0.9,
    generated_answers_path="./QGdata",
    no_cuda=False
)


from types import SimpleNamespace

args = SimpleNamespace(**question_generator_args)

## Generate

In [ ]:
import logging as logger
import os
from tqdm.notebook import tqdm, trange
from functools import partial

import torch
from torch.utils.data import TensorDataset, SequentialSampler, DataLoader, RandomSampler
from torch.optim import AdamW
from transformers import (
    T5Config,
    T5Tokenizer,
    T5ForConditionalGeneration,
    BertForQuestionAnswering,
    AutoModelForQuestionAnswering,
    BertTokenizerFast,
    AutoTokenizer,
    BertConfig,
    get_linear_schedule_with_warmup,
)

from torch.utils.tensorboard import SummaryWriter

logger.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logger.INFO
    )

if not os.path.exists(args.output_dir):
    os.makedirs(args.output_dir)

args.device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")


qg_config = T5Config.from_pretrained(args.qg_model)

qg_tokenizer = T5Tokenizer.from_pretrained(args.qg_model)
# qg_tokenizer.add_tokens(['<sep>'])

qg_model = T5ForConditionalGeneration.from_pretrained(
    args.qg_model,
    config=qg_config
)

qg_model.resize_token_embeddings(len(qg_tokenizer))
qg_model.eval()
qg_model.to(args.device)


qa_tokenizer = AutoTokenizer.from_pretrained(args.qa_model)
qa_model = AutoModelForQuestionAnswering.from_pretrained(args.qa_model)
qa_model.eval()
qa_model.to(args.device)

In [ ]:
f = open("./BARDContextandAnswer.json")
text_data = json.load(f)
f.close()

In [ ]:
text_data['data'][4333]

{'title': 'economy',
 'context': 'এনসিসি ব্যাংকের উদ্যোক্তা পরিচালক মো. নুরুন নেওয়াজ সেলিম তৃতীয়বারের মতো ব্যাংকের চেয়ারম্যান নির্বাচিত হয়েছেন। ব্যাংকের আরেক উদ্যোক্তা পরিচালক এ এস এম মাঈন উদ্দিন মোনেম ভাইস চেয়ারম্যান নির্বাচিত হয়েছেন।নুরুন নেওয়াজ চট্টগ্রাম চেম্বারের জ্যেষ্ঠ সহসভাপতি ও চট্টগ্রামের ফেনী জেলা সমিতির সভাপতি। তিনি ইলেক্ট্রোমার্ট ও ট্রেড ইন্টারন্যাশনাল মার্কেটিংয়ের চেয়ারম্যান এবং সেন্ট্রাল ইনস্যুরেন্স কোম্পানির সাবেক চেয়ারম্যান।মাঈনুদ্দিন মোনেম বর্তমানে আবদুল মোনেম লিমিটেডের উপব্যবস্থাপনা পরিচালক, নোভাস ফার্মাসিউটিক্যালস ও এএম সিকিউরিটিজ অ্যান্ড ফাইন্যান্সিয়াল সার্ভিসেসের প্রধান নির্বাহী। ।',
 'sent_list': ['এনসিসি ব্যাংকের উদ্যোক্তা পরিচালক মো. নুরুন নেওয়াজ সেলিম তৃতীয়বারের মতো ব্যাংকের চেয়ারম্যান নির্বাচিত হয়েছেন।',
  'ব্যাংকের আরেক উদ্যোক্তা পরিচালক এ এস এম মাঈন উদ্দিন মোনেম ভাইস চেয়ারম্যান নির্বাচিত হয়েছেন।',
  'নুরুন নেওয়াজ চট্টগ্রাম চেম্বারের জ্যেষ্ঠ সহসভাপতি ও চট্টগ্রামের ফেনী জেলা সমিতির সভাপতি।',
  'তিনি ইলেক্ট্রোমার্ট ও ট্রেড ইন্টারন্যাশনাল মার্ক

In [ ]:
ids = []
qg_data = []
ans_data = []
for di in range(len(text_data['data'])):
    data = text_data['data'][di]
    for answer in data['answers']:
        if len(data['sent_list'][answer['sent_with_ans']])>1 and len(answer['text'])>1:
            text = data['sent_list'][answer['sent_with_ans']]+' <sep> '+answer['text']
            qg_data.append(text)
            ans_data.append(answer)
            ids.append(di)

In [ ]:
# ids = []
# qg_data = []
# ans_data = []
# for di in range(len(text_data['data'])):
#     data = text_data['data'][di]
#     for sent in data['sent_list']:
#         if(len(sent)>1):
#             text = sent+' <sep> '+"impossible"
#             qg_data.append(text)
#             ans_data.append("impossible")
#             ids.append(di)

In [ ]:
import math
m = len(qg_data)
mini_batch_size = 32
mini_batches = []
num_complete_minibatches = math.floor(m / mini_batch_size) 
for k in range(0, num_complete_minibatches):

    mini_batch_X = qg_data[k*mini_batch_size:(k+1)*mini_batch_size]
    mini_batches.append(mini_batch_X)

if m % mini_batch_size != 0:
    mini_batch_X = qg_data[num_complete_minibatches*mini_batch_size:]
    mini_batches.append(mini_batch_X)

In [ ]:
questions = []
print('No. of batches: ', len(mini_batches))

for batch in tqdm(mini_batches):
  encodings = qg_tokenizer(batch, return_tensors="pt",max_length=512,truncation=True, padding=True)
  all_input_ids = torch.tensor(encodings['input_ids'], dtype=torch.long)
  outputs = qg_model.generate(all_input_ids.to(args.device),
                    max_length=50, 
                    num_beams=5, 
                    no_repeat_ngram_size=2, 
                    num_return_sequences=1, 
                    early_stopping=True)
  batch_ques = outputs.detach().cpu().numpy()
  batch_ques = batch_ques.reshape(len(batch),1,-1)
  
  for e in batch_ques:
    context_ques = []
    for aques in e:
      context_ques.append(qg_tokenizer.decode(aques, skip_special_tokens=True))
    questions.append(context_ques)
  

No. of batches:  6605


  0%|          | 0/6605 [00:00<?, ?it/s]

/tmp/ipykernel_10778/3621848040.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  all_input_ids = torch.tensor(encodings['input_ids'], dtype=torch.long)


In [ ]:
examples = []
for i in range(len(ids)):
    examples.append({'id': ids[i],
                        'context':text_data['data'][ids[i]]['context'],
                        'sent_with_ans': qg_data[i].split(" <sep> ")[0],
                        'questions':questions[i],
                        'answer': ans_data[i]
    })

In [ ]:
examples[102]

In [ ]:
temp_data = {}
temp_data['data'] =examples
final_out_file = open('./BARDQAtempDataset.json', "w")
json.dump(temp_data, final_out_file, indent = 4) # save whole data replace parts later
final_out_file.close()